In [1]:
import os
import sys

sys.path.append('..')
from models import *
import numpy as np
import pandas as pd
import matplotlib as plt
import sklearn
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from numpy import zeros
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import datasets
from sklearn.externals import joblib

le = LabelEncoder()

df = pd.read_hdf("../data_extract/features_dataset_2b.h5")

In [2]:
NUM_PEOPLE = 10
df = df.loc[df['id']<NUM_PEOPLE]

In [3]:
clist = ["chroma{}".format(i) for i in range(12)]
lclist = ["lc{}".format(i) for i in range(12)]
df[lclist] = np.log(df[clist])
flist = ["mfcc{}".format(i) for i in range(14)]+lclist+["centroid","crest","flatness","kurtosis","mean"]
df_train, df_cross, df_test = get_partitions(df)

In [4]:
obj = SVM(flist)
obj.fit(df_train)
joblib.dump(obj.model, 'SVM.pkl') 

['SVM.pkl']

In [6]:
clf = joblib.load('SVM.pkl') 
obj.predict_df(df_cross, clf)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        11
          1       1.00      1.00      1.00        15
          2       1.00      1.00      1.00         7
          3       1.00      1.00      1.00         9
          4       1.00      1.00      1.00        10

avg / total       1.00      1.00      1.00        52

